In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

#sessions = pd.read_excel("C:\\Users\\subha\\Appen\\PE Project Report - NWW_V2_1_Wakarusa\\9999_Falcon_Project_Wakarusa_MUL_MUL_collection_report.xlsx",  sheet_name='Sessions')
#files = pd.read_excel("C:\\Users\\subha\\Appen\\PE Project Report - NWW_V2_1_Wakarusa\\9999_Falcon_Project_Wakarusa_MUL_MUL_collection_report.xlsx",  sheet_name='Files')

sessions = pd.read_excel("C:\\Users\subha\\Desktop\\Appen\Wakewords2.1\\Data\\9999_Falcon_Project_Wakarusa_MUL_MUL_collection_report.xlsx",  sheet_name='Sessions')
files = pd.read_excel("C:\\Users\\subha\\Desktop\\Appen\\Wakewords2.1\\Data\\9999_Falcon_Project_Wakarusa_MUL_MUL_collection_report.xlsx",  sheet_name='Files')

print("data loaded in dataframe")


data loaded in dataframe


In [2]:
#dropping incomplete
#sessions = sessions.loc[sessions['completed'] == '']
sessions['completed'] = pd.to_datetime(sessions['completed'], errors='coerce')
sessions = sessions[sessions['completed'].notnull()]


sessions['input_ethnicity_it'] = ""
sessions['input_first_language_it'] = "" 

sessions['input_ethnicity_fr'] = ""
sessions['input_first_language_fr'] = "" 


#separating countries
USA_data = sessions.loc[sessions['input_country1'] == 'USA']
GBR_data = sessions.loc[sessions['input_country1'] == 'GBR']
AUS_data = sessions.loc[sessions['input_country1'] == 'AUS']
IT_data = sessions[sessions['input_state_it'].notnull()]
FR_data = sessions[sessions['input_state_fr'].notnull()]

#aligning corresponding cols to countries
USA_data_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent_usa","input_ethnicity_usa","input_first_language_usa","input_gender","input_home_language","input_hometown","input_learned_english","input_other_region_it","input_other_surroundings","input_state_usa","input_surroundings","input_years_in_us"]
GBR_data_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent_gbr","input_ethnicity_gbr","input_first_language_gbr","input_gender","input_home_language","input_hometown","input_learned_english","input_other_region_it","input_other_surroundings","input_state_gbr","input_surroundings","input_years_in_us"]
AUS_data_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent_aus","input_ethnicity_aus","input_first_language_aus","input_gender","input_home_language","input_hometown","input_learned_english","input_other_region_it","input_other_surroundings","input_state_aus","input_surroundings","input_years_in_us"]
IT_data_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent","input_ethnicity_it","input_first_language_it","input_gender","input_home_language","input_hometown","input_learned_italian","input_other_region_it","input_other_surroundings","input_state_it","input_surroundings","input_years_in_us"]
FR_data_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent","input_ethnicity_fr","input_first_language_fr","input_gender","input_home_language","input_hometown","input_learned_french","input_other_region_it","input_other_surroundings","input_state_fr","input_surroundings","input_years_in_us"]

USA_data = USA_data[USA_data_cols]
GBR_data = GBR_data[GBR_data_cols]
AUS_data = AUS_data[AUS_data_cols]
IT_data = IT_data[IT_data_cols]
FR_data = FR_data[FR_data_cols]

#Columned renamed for all to make it common
common_cols = ["directory_name","pin","script_name","script_number","total_items","recorded_items","accepted_items","rejected_items","total_duration","date","completed","abandoned","email","input_Current_hometown","input_Mask","input_Socioeconomic_bkgd","input_age","input_age_english","input_ambient","input_city","input_country1","input_date1","input_education","input_english_accent","input_ethnicity","input_first_language","input_gender","input_home_language","input_hometown","input_learned","input_other_region","input_other_surroundings","input_state","input_surroundings","input_years_in"]

countries_name = [USA_data,GBR_data,AUS_data,IT_data,FR_data]
for cntry in countries_name:
    cntry.columns = common_cols
stacked_data = pd.concat(countries_name, ignore_index=True)


#FR and IT country data filling
stacked_data.loc[stacked_data['script_name'].str.contains('it', case=False), 'input_country1'] = 'ITA'
stacked_data.loc[stacked_data['script_name'].str.contains('fr', case=False), 'input_country1'] = 'FRA'
stacked_data['member_id'] = stacked_data['pin']
#merging files data with session data
stacked_data = pd.merge(files,stacked_data, how='left', on = 'directory_name')
#dropping blank rows
stacked_data = stacked_data.loc[stacked_data['member_id'].notnull()]
#sorting
stacked_data=stacked_data.sort_values(['directory_name', 'promptnum'], ascending=[True, True])
#creaing required cols
stacked_data["prompt_text"] = stacked_data["Identifier"]+", "+ stacked_data["Utts"]
stacked_data['style'] = "Scripted"
stacked_data['scope'] = "in domain"
stacked_data["project"] = ""
stacked_data["locale"] = ""
stacked_data.loc[stacked_data['input_country1'].str.contains('USA', case=False), 'locale'] = 'en_US'
stacked_data.loc[stacked_data['input_country1'].str.contains('GBR', case=False), 'locale'] = 'en_GB'
stacked_data.loc[stacked_data['input_country1'].str.contains('AUS', case=False), 'locale'] = 'en_AU'
stacked_data.loc[stacked_data['input_country1'].str.contains('IT', case=False), 'locale'] = 'it_IT'
stacked_data.loc[stacked_data['input_country1'].str.contains('FR', case=False), 'locale'] = 'fr_FR'

stacked_data.loc[stacked_data['input_country1'].str.contains('USA', case=False), 'project'] = 'T154939089_New_WakeWords_2.1_EN_US'
stacked_data.loc[stacked_data['input_country1'].str.contains('GBR', case=False), 'project'] = 'T154939089_New_WakeWords_2.1_EN_GB'
stacked_data.loc[stacked_data['input_country1'].str.contains('AUS', case=False), 'project'] = 'T154939089_New_WakeWords_2.1_EN_AU'
stacked_data.loc[stacked_data['input_country1'].str.contains('IT', case=False), 'project'] = 'T154939089_New_WakeWords_2.1_IT_IT'
stacked_data.loc[stacked_data['input_country1'].str.contains('FR', case=False), 'project'] = 'T154939089_New_WakeWords_2.1_FR_FR'

stacked_data[['ambi1','ambi2', 'ambi3', 'ambi4']] = stacked_data['input_ambient'].str.split('_', expand=True)
stacked_data["environment"] = stacked_data['ambi1']+"_"+stacked_data['ambi2']+"_"+stacked_data['ambi3']
stacked_data["noise_type"] = stacked_data['ambi4']
now = datetime.now()
dt_string = now.strftime("%Y%m%d")
stacked_data["vendor_ds"] = dt_string #Current date 
stacked_data["bluetooth"] = ""
stacked_data["transcription"] = ""
stacked_data["english_accent_locale"] = ""
stacked_data["conversation_id"] = "" #stacked_data['member_id']+"_"+stacked_data['promptnum'].astype(str)
stacked_data["biometric_consent"] = "No"
stacked_data['SNRdB'] = ""
stacked_data['BkgRmsdB'] = ""
stacked_data['SigRMSdB'] = ""
stacked_data['SigthreshdB'] = ""
stacked_data['child_language'] = "n/a"
stacked_data['Topic'] = stacked_data['Identifier']

#renaming cols
col_map={'Subtitle' : 'subtitle',
'Topic' : 'topic',
'member_id' : 'member_id',
'prompt_text' : 'prompt_text',
'file' : 'audio_filename',
'input_gender' : 'gender',
'input_age' : 'age',
'input_city' : 'city',
'input_state' : 'state',
'input_age_english' : 'age_english',
'input_first_language' : 'first_language',
'input_home_language' : 'home_language',
'input_years_in' : 'years_in_us',
'style' : 'style',
'environment' : 'environment',
'vendor_ds' : 'vendor_ds',
'scope' : 'scope',
'project' : 'project',
'locale' : 'locale',
'input_education' : 'education',
'bluetooth' : 'bluetooth',
'noise_type' : 'noise_type',
'transcription' : 'transcription',
'input_country1' : 'country',
'input_surroundings' : 'environment_description',
'input_other_surroundings' : 'environment_description_other',
'input_Mask' : 'face_mask',
'SNRdB' : 'SNRdB',
'BkgRmsdB' : 'BkgRmsdB',
'SigRMSdB' : 'SigRMSdB',
'SigthreshdB' : 'SigthreshdB',
'input_Socioeconomic_bkgd' : 'Socioeconomic_bkgd',
'input_hometown' : 'Hometown',
'input_ethnicity' : 'Ethnicity',
'input_Current_hometown' : 'Current_hometown',
'child_language': 'child_language',
'input_english_accent' : 'english_accent',
'english_accent_locale' : 'english_accent_locale',
'conversation_id' : 'conversation_id',
'biometric_consent' : 'biometric_consent'
}
stacked_data = stacked_data.rename(columns=col_map)

#cleaning cols
gender_val = {
"Uomo":"Male",
"Homme":"Male",
"Donna":"Female",
"Femme":"Female",
"Non Binario":"Non-Binary",
"Non-binaire":"Non-Binary",
}
stacked_data['gender'] = stacked_data['gender'].replace(gender_val)
city_val = {
"rurale":"Rural",
"urbano":"Urban",
"suburbano":"Suburban",
"Campagne":"Rural",
"Ville de banlieue":"Suburban",
"Aire urbaine":"Urban"
}
stacked_data['city'] = stacked_data['city'].replace(city_val)

stacked_data.loc[stacked_data['script_name'].str.contains('it', case=False), 'first_language'] = 'Italian'
stacked_data.loc[stacked_data['script_name'].str.contains('fr', case=False), 'first_language'] = 'French'

mask_val = {
    "Non":"No",
    "Oui":"Yes",
    "Si":"Yes"
}
stacked_data['face_mask'] = stacked_data['face_mask'].replace(mask_val)

envsur_val = {
"I: A casa con la TV / elettrodomestico accesi":"I: Home with TV/noisy appliance on",
"I: A casa con persone che parlano/bambini":"I: Home with people talking/kids",
"I: All'Al chiuso di luoghi pubblici":"I: Inside noisy public buildings/places",
"I: A casa senza molto rumore":"I: Home with not much noise",
"I: Altro":"I: Other",
"I: A la maison avec une TV ou des appareils électroménagers allumés":"I: Home with TV/noisy appliance on",
"I: A la maison avec des gens qui parlent/des enfants":"I: Home with people talking/kids",
"I: Dans un lieu public en intérieur":"I: Inside noisy public buildings/places",
"I: A la maison sans trop de bruit":"I: Home with not much noise",
"I: Autre":"I: Other"
}
stacked_data['environment_description'] = stacked_data['environment_description'].str.strip()
stacked_data['environment_description'] = stacked_data['environment_description'].replace(envsur_val)

env_val = {
"Al chiuso_rumoroso_alto":"indoor_noisy_loud",
"Al chiuso_rumoroso_normale":"indoor_noisy_normal",
"Al chiuso_rumoroso_basso":"indoor_noisy_lowvolume",
"Al chiuso_tranquillo_alto":"indoor_quiet_loud",
"Al chiuso_tranquillo_normale":"indoor_quiet_normal",
"Al chiuso_tranquillo_basso":"indoor_quiet_lowvolume",
    
"interieur_bruyant_fort":"indoor_noisy_loud",
"interieur_bruyant_volumenormal":"indoor_noisy_normal",
"interieur_bruyant_volumefaible":"indoor_noisy_lowvolume",
"interieur_calme_volumefort":"indoor_quiet_loud",
"interieur_calme_volumenormal":"indoor_quiet_normal",
"interieur_calme_volumefaible":"indoor_quiet_lowvolume",
    
"interieur_bruyant_volumefort":"indoor_noisy_loud",    
}
stacked_data['environment'] =stacked_data['environment'].str.strip()
stacked_data['environment'] = stacked_data['environment'].replace(env_val)

noise_val = {
"umano":"Human Speech",
"ambientale":"Ambient Speech",
"non parlato":"Non-Speech",
"voix humaine":"Human Speech",
"voixbruitdefond":"Ambient Speech",
"aucunevoixhumaine":"Non-Speech",

"voixhumaine":"Human Speech", 
"ambient speech":"Ambient Speech",
"human speech":"Human Speech", 
"non-speech":"Non-Speech",    
}
stacked_data['noise_type'] =stacked_data['noise_type'].str.strip()
stacked_data['noise_type'] = stacked_data['noise_type'].replace(noise_val)

sco_val = {
            "< 22 000 euros":"Low",
            "22 000 a 50 000 euros":"Medium",
            "50 000 euros et +":"Affluent",
            "< 21k €":"Low",
            "21k € - 48k €":"Medium",
            "> 48k+ €":"Affluent"    
}
stacked_data['Socioeconomic_bkgd'] = stacked_data['Socioeconomic_bkgd'].replace(sco_val)

edu_val={
#ITA
"Dottorato":"Doctorate",
"Laurea magistrale":"Masters Degree",
"Master":"Masters Degree",
"Laruea triennale":"Bachelor Degree",
"Studi universitari in corso":"Some College",
"Diploma scuola superiore":"High School",
"Diploma scuola media":"Middle School",
"Diploma scuola elementare":"Elementary School",
"Altro":"Other",
#FRA    
"DUT/BTS/BAC+2":"Associates Degree",
"Licence/BAC+3":"Bachelors Degree",
"En etudes superieures actuellement":"Some College",
"Doctorat":"Doctorate",
"Ecole primaire":"Elementary School",
"Lycee":"High School",
"Master/BAC+5":"Masters Degree",
"College/Brevet":"Some College",
"Ai eu des cours a l'universite sans terminer":"Some College",
"Autre":"Other",
    #eng
"Doctorate":"Doctorate",
"Masters_degree":"Masters Degree",
"Bachelor_degree": "Bachelors Degree",
"Associate_degree": "Associates Degree" ,
"Some_college":"Some College",
"High_school":"High School",
"Middle_school":"Middle School",
"Elementary_school": "Elementary School"
}
stacked_data['education'] = stacked_data['education'].replace(edu_val)


acc_val ={
    
"Alsace, Aquitaine, Auvergne, Brittany, Burgundy, Centre-Val de Loire, Champagne- Ardenne, Franche-Comté, Île-de-France,":"Alsace, Aquitaine, Auvergne, Brittany, Burgundy, Centre-Val de Loire, Champagne- Ardenne, Franche-Comté, Île-de-France, Languedoc-Roussillon, Limousin, Lorraine, Lower Normandy, Midi-Pyrénées, Nord-Pas-de-Calais, Pays de la Loire, Picardy, Poitou-Charentes, Provence-Alpes-Côte d'Azur, Rhône-Alpes, Upper Normandy",
"Languedoc-Roussillon, Limousin, Lorraine, Lower Normandy, Midi-Pyrénées, Nord-Pas-de-Calais, Pays de la Loire, Picardy, Poitou-Charentes, Provence-Alpes-Côte d'Azur, Rhône-Alpes, Upper Normandy ":"Alsace, Aquitaine, Auvergne, Brittany, Burgundy, Centre-Val de Loire, Champagne- Ardenne, Franche-Comté, Île-de-France, Languedoc-Roussillon, Limousin, Lorraine, Lower Normandy, Midi-Pyrénées, Nord-Pas-de-Calais, Pays de la Loire, Picardy, Poitou-Charentes, Provence-Alpes-Côte d'Azur, Rhône-Alpes, Upper Normandy",
"Languedoc-Roussillon, Limousin, Lorraine, Lower Normandy, Midi-Pyrénées, Nord-Pas-de-Calais, Pays de la Loire, Picardy, Poitou-Charentes, Provence-Alpes-Côte d":"Alsace, Aquitaine, Auvergne, Brittany, Burgundy, Centre-Val de Loire, Champagne- Ardenne, Franche-Comté, Île-de-France, Languedoc-Roussillon, Limousin, Lorraine, Lower Normandy, Midi-Pyrénées, Nord-Pas-de-Calais, Pays de la Loire, Picardy, Poitou-Charentes, Provence-Alpes-Côte d'Azur, Rhône-Alpes, Upper Normandy",

"Canadian French (Quebecois, Acadian, Franco-Ontarian)":"Canadian French (Quebecois, Acadian, Franco-Ontarian)",  
"Dialects from former French colonies in Africa (DR of Congo, Ivory Coast, Senegal, Morocco, Algeria, Mali, Guinea, Niger, Nigeria, Chad, Madagascar) ":"Dialects from former French colonies in Africa (DR of Congo, Ivory Coast, Senegal, Morocco, Algeria, Mali, Guinea, Niger, Nigeria, Chad, Madagascar)", 
"Autres (Haiti, United States (Cajun), Laos, Cambodia, Vietnam, Lebanon) ":"Other (Haiti, United States (Cajun), Laos, Cambodia, Vietnam, Lebanon)" 
}
stacked_data['english_accent'] = stacked_data['english_accent'].replace(acc_val)

#Excluding already delivered data
delivered_data = pd.read_excel("C:\\Users\\subha\\Appen\\Falcon Speech Collection 2023 - NWW 2.1\\tracker\\Sources\\delivery_report.xlsx")
# Find the intersection of the two dataframes
intersection = stacked_data[stacked_data['directory_name'].isin(delivered_data['directory_name'])]
stacked_data = stacked_data.drop(intersection.index)

#final cols selected
col_final = ["subtitle","topic","member_id","prompt_text","audio_filename","gender","age","city","state",
             "age_english","first_language","home_language","years_in_us","style","environment","vendor_ds",
             "scope","project","locale","education","bluetooth","noise_type","transcription","country",
             "environment_description","environment_description_other","face_mask","SNRdB","BkgRmsdB",
             "SigRMSdB","SigthreshdB","Socioeconomic_bkgd","Hometown","Ethnicity","Current_hometown","child_language",
             "english_accent","english_accent_locale","conversation_id","biometric_consent"]
output_data = stacked_data[col_final]

# Remove text after , / . (
def remove_text(text):
    for char in [',', '/', '.', '(']:
        text = text.split(char)[0]
    return text

# Check the type of each value in the column1 column
def check_type(text):
    if type(text) == float:
        return str(text)
    else:
        return text

output_data['Hometown'] = output_data['Hometown'].apply(check_type)
output_data['Current_hometown'] = output_data['Current_hometown'].apply(check_type)

output_data['Hometown'] = output_data['Hometown'].apply(remove_text)
output_data['Current_hometown'] = output_data['Current_hometown'].apply(remove_text)

stacked_data.to_excel("C:\\Users\\subha\\Desktop\\Appen\\Wakewords2.1\\Data\\staked_output.xlsx", index=False)
output_data.to_excel("C:\\Users\\subha\\Desktop\\Appen\\Wakewords2.1\\Data\\output_data.xlsx", index=False)

print('done')


C:\Users\subha\AppData\Local\Temp/ipykernel_9416/1532429656.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_data['Hometown'] = output_data['Hometown'].apply(check_type)
C:\Users\subha\AppData\Local\Temp/ipykernel_9416/1532429656.py:300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_data['Current_hometown'] = output_data['Current_hometown'].apply(check_type)
C:\Users\subha\AppData\Local\Temp/ipykernel_9416/1532429656.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy o

done


In [6]:
import pandas as pd
sessions = pd.read_excel("C:\\Users\\subha\\Appen\\Falcon Speech Collection 2023 - NWW 2.1\\package\\20230721\\output_data_20230721.xlsx",  sheet_name='Sheet1')

USA_data = sessions.loc[sessions['country'] == 'USA']
GBR_data = sessions.loc[sessions['country'] == 'GBR']
AUS_data = sessions.loc[sessions['country'] == 'AUS']
IT_data = sessions[sessions['country'] == 'ITA']
FR_data = sessions[sessions['country'] == 'FRA']

countries_name = [USA_data,GBR_data,AUS_data,IT_data,FR_data]
nameC=["EN_US","EN_GB","EN_AU","IT_IT","FR_FR"]
date="20230721"
for x,y in zip(countries_name,nameC):
    x.to_excel("C:\\Users\\subha\\Appen\\Falcon Speech Collection 2023 - NWW 2.1\\package\\{}\\Appen_VoiceDataCollectionServices_{}_New_WakeWords_2_1_{}.xlsx".format(date,date,y), index=False)
